In [3]:
import pandas as pd
from datetime import datetime, timedelta
        

In [4]:
df = pd.read_csv('input.csv')

In [5]:
date_columns = ['Compensation 1 date', 'Compensation 2 date', 'Review 1 date', 'Review 2 date',
                'Engagement 1 date', 'Engagement 2 date']

# List of performance rating columns
performance_rating_columns = ['Review 1', 'Review 2']

# List of engagement score columns
engagement_score_columns = ['Engagement 1', 'Engagement 2']

# Create a list to store transformed data
transformed_data = []

In [6]:
for index, row in df.iterrows():
    effective_dates = [pd.to_datetime(row[col], errors='coerce') for col in date_columns]
    effective_dates = [date for date in effective_dates if not pd.isnull(date)]
    effective_dates.sort()

    # Create historical records based on effective dates
    for i in range(len(effective_dates)):
        record = row.copy()
        record['Effective Date'] = effective_dates[i]
        
        # Calculate End Date
        if i < len(effective_dates) - 1:
            record['End Date'] = effective_dates[i + 1] - timedelta(days=1)
        else:
            record['End Date'] = datetime(2100, 1, 1)

        # Match performance rating with date and create a new column
        for rating_col in performance_rating_columns:
            rating_date = pd.to_datetime(row[f'{rating_col} date'], errors='coerce')
            if not pd.isnull(row[rating_col]) and rating_date <= effective_dates[i]:
                record['Performance Rating'] = row[rating_col]
        
        # Match engagement score with date and create a new column
        for score_col in engagement_score_columns:
            score_date = pd.to_datetime(row[f'{score_col} date'], errors='coerce')
            if not pd.isnull(row[score_col]) and score_date <= effective_dates[i]:
                record['Engagement Score'] = row[score_col]

        transformed_data.append(record)

# Convert the list of dictionaries to a new DataFrame
transformed_df = pd.DataFrame(transformed_data)

# Identify the last compensation for each employee and create a new 'Last Compensation' column
last_compensation = df[['Employee Code', 'Compensation 2']].groupby('Employee Code')['Compensation 2'].last().reset_index()
last_compensation.rename(columns={'Compensation 2': 'Last Compensation'}, inplace=True)

# Combine the last compensation with the transformed data
transformed_df = pd.merge(transformed_df, last_compensation, on='Employee Code', how='left')

# Create a new 'Compensation' column based on the available compensation columns
transformed_df['Compensation'] = transformed_df.apply(lambda row: row['Compensation 2'] 
                                                       if not pd.isnull(row['Compensation 2']) 
                                                        else (row['Compensation 1'] 
                                                              if not pd.isnull(row['Compensation 1']) 
                                                              else row['Effective Date']), 
                                                        axis=1)
transformed_df = transformed_df.drop_duplicates()

In [7]:
transformed_df.head(20)

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,...,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date,Effective Date,End Date,Engagement Score,Performance Rating,Last Compensation
0,2,1.0,2021-01-01,NaN,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2021-03-01,2021-05-31,4.0,NaN,20000.0
1,2,1.0,2021-01-01,NaN,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2021-06-01,2021-12-31,4.0,9.0,20000.0
2,2,1.0,2021-01-01,NaN,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2022-01-01,2022-02-28,4.0,9.0,20000.0
3,2,1.0,2021-01-01,NaN,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2022-03-01,2022-05-31,5.0,9.0,20000.0
4,2,1.0,2021-01-01,NaN,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2022-06-01,2022-12-31,5.0,9.5,20000.0
5,2,1.0,2021-01-01,NaN,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2023-01-01,2100-01-01,5.0,9.5,20000.0
6,3,1.0,2021-01-01,2023-12-31,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2021-03-01,2021-05-31,4.0,NaN,20000.0
7,3,1.0,2021-01-01,2023-12-31,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2021-06-01,2021-12-31,4.0,9.0,20000.0
8,3,1.0,2021-01-01,2023-12-31,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2022-01-01,2022-02-28,4.0,9.0,20000.0
9,3,1.0,2021-01-01,2023-12-31,20000.0,10000.0,2022-01-01,20000.0,2023-01-01,9.0,...,2022-06-01,4.0,2021-03-01,5.0,2022-03-01,2022-03-01,2022-05-31,5.0,9.0,20000.0


In [8]:
columns_to_remove = ['Date of Exit','Review 1','Review 2','Engagement 1','Engagement 2','Last Compensation','Compensation','Compensation 1','Compensation 1','Compensation 2','Compensation 1 date', 'Compensation 2 date', 'Date of Joining',
                      'Review 1 date', 'Review 2 date', 'Engagement 1 date', 'Engagement 2 date']

# Drop specified columns
df = transformed_df.drop(columns=columns_to_remove)

In [9]:
df.head(20)

,Employee Code,Manager Employee Code,Effective Date,End Date,Engagement Score,Performance Rating
0,2,1.0,2021-03-01,2021-05-31,4.0,NaN
1,2,1.0,2021-06-01,2021-12-31,4.0,9.0
2,2,1.0,2022-01-01,2022-02-28,4.0,9.0
3,2,1.0,2022-03-01,2022-05-31,5.0,9.0
4,2,1.0,2022-06-01,2022-12-31,5.0,9.5
5,2,1.0,2023-01-01,2100-01-01,5.0,9.5
6,3,1.0,2021-03-01,2021-05-31,4.0,NaN
7,3,1.0,2021-06-01,2021-12-31,4.0,9.0
8,3,1.0,2022-01-01,2022-02-28,4.0,9.0
9,3,1.0,2022-03-01,2022-05-31,5.0,9.0


In [11]:
df.to_csv('output.csv', index=False)